# Virtual Try-On System (VITON)

## Step 1: Install Required Libraries
- Ensure you have the necessary libraries installed, such as `torch`, `torchvision`, and `opencv-python`.

In [1]:
pip install torch torchvision opencv-python

   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/199.4 MB 1.6 MB/s eta 0:02:05
   ---------------------------------------- 0.5/199.4 MB 6.2 MB/s eta 0:00:32
   ---------------------------------------- 0.9/199.4 MB 7.3 MB/s eta 0:00:28
   ---------------------------------------- 1.4/199.4 MB 8.3 MB/s eta 0:00:24
   ---------------------------------------- 1.9/199.4 MB 8.7 MB/s eta 0:00:23
   ---------------------------------------- 2.3/199.4 MB 9.3 MB/s eta 0:00:22
    --------------------------------------- 2.8/199.4 MB 9.0 MB/s eta 0:00:22
    --------------------------------------- 3.3/199.4 MB 9.3 MB/s eta 0:00:22
    --------------------------------------- 3.9/199.4 MB 9.6 MB/s eta 0:00:21
    --------------------------------------- 4.4/199.4 MB 9.7 MB/s eta 0:00:21
    --------------------------------------- 5.0/199.4 MB 9.9 MB/s eta 0:00:20
   - -------------------------------------- 5.5/199.4 MB 10.0 MB/s eta 

## Step 2: Import Libraries
- Import the required libraries at the beginning of your Python script.

In [3]:
import cv2
import numpy as np
import torch
from torchvision import models, transforms

## Step 3: Load the Pre-trained Segmentation Model
- Load a pre-trained segmentation model (e.g., DeepLabV3) for segmenting clothing from images.

In [4]:
# Load pre-trained segmentation model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.eval()

C:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ASUS\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\deeplabv3_resnet101_coco-586e9e4e.pth
100%|███████████████████████████████████████████████████████████████████████████████| 233M/233M [00:21<00:00, 11.6MB/s]


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

## Step 4: Define Image Preprocessing Function
- Create a function to preprocess the input images, ensuring they are in the right format for the model. This typically involves resizing, normalizing, and converting images to tensors.

In [5]:
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),  # Resize to the model's input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

## Step 5: Define Clothing Segmentation Function
- Create a function that uses the model to segment the clothing from the input image. This function should preprocess the image, perform a forward pass through the model, and return the segmented output.

In [6]:
def segment_clothing(image):
    input_tensor = preprocess_image(image)  # Preprocess the input image
    with torch.no_grad():  # Disable gradient calculation for inference
        output = model(input_tensor)['out'][0]  # Forward pass through the model
    segmented = output.argmax(0).byte().cpu().numpy()  # Get the segmented output
    return segmented

## Step 6: Define Clothing Application Function
- This function applies the segmented clothing to the model image. It typically involves creating a mask from the segmentation result and blending the clothing onto the model image.

In [7]:
def apply_clothing_to_model(clothing_segmentation, model_image):
    # Create a mask from the segmentation result
    clothing_mask = cv2.cvtColor(clothing_segmentation, cv2.COLOR_GRAY2BGR)  # Convert to 3 channels
    clothing_mask = clothing_mask.astype(float) / 255.0  # Normalize mask to [0, 1]

    # Resize the model image to match the clothing mask
    model_image_resized = cv2.resize(model_image, (clothing_segmentation.shape[1], clothing_segmentation.shape[0]))

    # Apply the clothing to the model image using the mask
    result = clothing_mask * model_image_resized + (1 - clothing_mask) * model_image_resized
    return (result * 255).astype(np.uint8)  # Convert back to uint8

## Step 7: Load Input Images
- Load the model image (the person) and the clothing image (the garment to be tried on) using OpenCV. Adjust the file paths as necessary.

In [10]:
# Load images
model_image = cv2.imread(r'C:\Users\ASUS\Downloads\images.jpeg')  # Replace with your model image path
clothing_image = cv2.imread(r'C:\Users\ASUS\Downloads\download.jpeg')  # Replace with your clothing image path

## Step 8: Segment Clothing
- Use the segmentation function on the clothing image to obtain the clothing segmentation mask.

In [11]:
# Segment clothing
segmented_clothing = segment_clothing(clothing_image)

## Step 9: Apply Clothing to Model
- Use the clothing application function to overlay the segmented clothing onto the model image.

In [12]:
# Apply clothing to model
result = apply_clothing_to_model(segmented_clothing, model_image)

## Step 10: Show Result
- Display the resulting image using OpenCV to visualize the virtual try-on effect.

In [14]:
# Show result
cv2.imshow('Virtual Try-On', result)
cv2.waitKey(0)
cv2.destroyAllWindows()